In [1]:
import pandas as pd
import numpy as np
import nibabel as ni
import scipy.stats as ss

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [3]:
mem_ref = np.array(ni.load("memory_network_T_map.nii").dataobj)

In [26]:
mem_df = pd.read_csv("year_mem.csv")
ids = list(mem_df["study_id"])

In [32]:
NDS_all=[]
NDS_pos_only=[]
NDS_neg_only=[]

NDS_all_avg=[]
NDS_pos_only_avg=[]
NDS_neg_only_avg=[]

In [28]:
for i in ids:
    try:
        nw = np.array(ni.load("brain_lesion_networks/"+i+"_A_lesion_ifhfix_func_seed_T.nii").dataobj)
    except:
        nw = np.array(ni.load("brain_lesion_networks/"+i+"_A_lesion_func_seed_T.nii").dataobj)
    tot = []
    pos = []
    neg = []
    for x in range(len(nw)):
        for y in range(len(nw[x])):
            for z in range(len(nw[x][y])):
                if mem_ref[x, y, z] != 0:
                    tot.append(nw[x, y, z])
                    if nw[x, y, z] > 0:
                        pos.append(nw[x, y, z])
                    if nw[x, y, z] < 0:
                        neg.append(nw[x, y, z])
    NDS_all.append(np.sum(np.array(tot)))
    NDS_pos_only.append(np.sum(np.array(pos)))
    NDS_neg_only.append(np.sum(np.array(neg)))
    NDS_all_avg.append(np.mean(np.array(tot)))
    if len(pos) > 0:
        NDS_pos_only_avg.append(np.mean(np.array(pos)))
    else:
        NDS_pos_only_avg.append(0)
    if len(neg) > 0:
        NDS_neg_only_avg.append(np.mean(np.array(neg)))
    else:
        NDS_neg_only_avg.append(0)

In [33]:
for i in ids:
    try:
        lesion = np.array(ni.load("brain_lesions/"+i+"_A_lesion_ifhfix.nii").dataobj)
    except:
        lesion = np.array(ni.load("brain_lesions/"+i+"_A_lesion.nii").dataobj)
    tot = []
    pos = []
    neg = []
    for x in range(len(lesion)):
        for y in range(len(lesion[x])):
            for z in range(len(lesion[x][y])):
                if lesion[x, y, z] != 0:
                    tot.append(mem_ref[x, y, z])
                    if mem_ref[x, y, z] > 0:
                        pos.append(mem_ref[x, y, z])
                    if mem_ref[x, y, z] < 0:
                        neg.append(mem_ref[x, y, z])
    NDS_all.append(np.sum(np.array(tot)))
    NDS_pos_only.append(np.sum(np.array(pos)))
    NDS_neg_only.append(np.sum(np.array(neg)))
    NDS_all_avg.append(np.mean(np.array(tot)))
    if len(pos) > 0:
        NDS_pos_only_avg.append(np.mean(np.array(pos)))
    else:
        NDS_pos_only_avg.append(0)
    if len(neg) > 0:
        NDS_neg_only_avg.append(np.mean(np.array(neg)))
    else:
        NDS_neg_only_avg.append(0)

In [34]:
mem_df["mem_NDS_true_tot"] = NDS_all
mem_df["mem_NDS_true_pos_only"] = NDS_pos_only
mem_df["mem_NDS_true_neg_only"] = NDS_neg_only

mem_df["mem_NDS_true_tot_avg"] = NDS_all_avg
mem_df["mem_NDS_true_pos_avg"] = NDS_pos_only_avg
mem_df["mem_NDS_true_neg_avg"] = NDS_neg_only_avg

In [29]:
mem_df["mem_NDS_tot"] = NDS_all
mem_df["mem_NDS_pos_only"] = NDS_pos_only
mem_df["mem_NDS_neg_only"] = NDS_neg_only

mem_df["mem_NDS_tot_avg"] = NDS_all_avg
mem_df["mem_NDS_pos_avg"] = NDS_pos_only_avg
mem_df["mem_NDS_neg_avg"] = NDS_neg_only_avg

In [30]:
mem_corr = []
for i in ids:
    try:
        nw = np.array(ni.load("brain_lesion_networks/"+i+"_A_lesion_ifhfix_func_seed_T.nii").dataobj)
    except:
        nw = np.array(ni.load("brain_lesion_networks/"+i+"_A_lesion_func_seed_T.nii").dataobj)
    ind_corr = []
    ref_corr = []
    for x in range(len(nw)):
        for y in range(len(nw[x])):
            for z in range(len(nw[x][y])):
                if mem_ref[x, y, z] != 0:
                    ind_corr.append(nw[x, y, z])
                    ref_corr.append(mem_ref[x, y, z])
    mem_corr.append(ss.pearsonr(ind_corr, ref_corr)[0])
mem_df["mem_sp_corr"] = mem_corr

In [35]:
mem_df.to_csv("year_mem.csv", index=False)